In [44]:
from google.colab import userdata


In [45]:
!pip install pandas openai tqdm tenacity scikit-learn tiktoken python-dotenv seaborn --upgrade --quiet

In [46]:

import json
import os
import time

import pandas as pd
!pip install openai
import tiktoken
import seaborn as sns
from tenacity import retry, wait_exponential
from tqdm import tqdm
from collections import defaultdict
import numpy as np
import openai
import matplotlib.pyplot as plt
import numpy as np
import warnings
warnings.filterwarnings('ignore')

tqdm.pandas()

openai.api_key = userdata.get('OpenAI')


In [47]:
os.environ["QDRANT_URL"] = "https://xxx.cloud.qdrant.io:6333"
os.environ["QDRANT_API_KEY"] = userdata.get('Qdrant')

In [48]:
pip install transformers

In [49]:
pip install tensorflow-text


In [50]:
pip install datasets


In [51]:
from datasets import load_dataset

dataset = load_dataset("empathetic_dialogues")
dataset

DatasetDict({
    train: Dataset({
        features: ['conv_id', 'utterance_idx', 'context', 'prompt', 'speaker_idx', 'utterance', 'selfeval', 'tags'],
        num_rows: 76673
    })
    validation: Dataset({
        features: ['conv_id', 'utterance_idx', 'context', 'prompt', 'speaker_idx', 'utterance', 'selfeval', 'tags'],
        num_rows: 12030
    })
    test: Dataset({
        features: ['conv_id', 'utterance_idx', 'context', 'prompt', 'speaker_idx', 'utterance', 'selfeval', 'tags'],
        num_rows: 10943
    })
})

In [52]:
dataset.set_format('pandas')
df_train = dataset['train'][:8000]
df_train=df_train.drop('conv_id',axis=1)
df_train=df_train.drop('utterance_idx',axis=1)
df_train=df_train.drop('speaker_idx',axis=1)
df_train=df_train.drop('selfeval',axis=1)
df_train=df_train.drop('tags',axis=1)
df_train

,context,prompt,utterance
0,sentimental,I remember going to the fireworks with my best...,I remember going to see the fireworks with my ...
1,sentimental,I remember going to the fireworks with my best...,Was this a friend you were in love with_comma_...
2,sentimental,I remember going to the fireworks with my best...,This was a best friend. I miss her.
3,sentimental,I remember going to the fireworks with my best...,Where has she gone?
4,sentimental,I remember going to the fireworks with my best...,We no longer talk.
...,...,...,...
7995,faithful,I am faithful to my girlfriend all the time_co...,oh that's bad. I feel sorry for you.
7996,anxious,Back when I was in school_comma_ i studied rea...,I think small things make me worried
7997,anxious,Back when I was in school_comma_ i studied rea...,Small things like what?
7998,anxious,Back when I was in school_comma_ i studied rea...,Back in school_comma_ I studied a lot for an i...


In [53]:
training_data = []

system_message = "You are an empathetic assistant. You are to generate empathetic responses to prompts."

def create_user_message(row):
    return f"""Title: {row['context']}\n\nprompt: {row['prompt']}\n\nutterance: {row['utterance']} """

def prepare_example_conversation(row):
    messages = []
    messages.append({"role": "system", "content": system_message})

    user_message = create_user_message(row)
    messages.append({"role": "user", "content": user_message})

    messages.append({"role": "assistant", "content": row["prompt"]})

    return {"messages": messages}

print(prepare_example_conversation(df_train.iloc[0]))

{'messages': [{'role': 'system', 'content': 'You are an empathetic assistant. You are to generate empathetic responses to prompts.'}, {'role': 'user', 'content': 'Title: sentimental\n\nprompt: I remember going to the fireworks with my best friend. There was a lot of people_comma_ but it only felt like us in the world.\n\nutterance: I remember going to see the fireworks with my best friend. It was the first time we ever spent time alone together. Although there was a lot of people_comma_ we felt like the only people in the world. '}, {'role': 'assistant', 'content': 'I remember going to the fireworks with my best friend. There was a lot of people_comma_ but it only felt like us in the world.'}]}


In [54]:
df_validation = dataset['validation'][:1000]
df_validation=df_validation.drop('conv_id',axis=1)
df_validation=df_validation.drop('utterance_idx',axis=1)
df_validation=df_validation.drop('speaker_idx',axis=1)
df_validation=df_validation.drop('selfeval',axis=1)
df_validation=df_validation.drop('tags',axis=1)
df_validation

,context,prompt,utterance
0,terrified,Today_comma_as i was leaving for work in the m...,Today_comma_as i was leaving for work in the m...
1,terrified,Today_comma_as i was leaving for work in the m...,Are you fine now?
2,terrified,Today_comma_as i was leaving for work in the m...,Yeah_comma_i'm doing alright now_comma_ but wi...
3,terrified,Today_comma_as i was leaving for work in the m...,Cool :) Is your car damaged a lot?
4,terrified,Today_comma_as i was leaving for work in the m...,The car was badly damaged_comma_i veered outsi...
...,...,...,...
995,embarrassed,I feel down in front of my students during games.,Oh really? thats sad_comma_ why is that?
996,embarrassed,I feel down in front of my students during games.,I tripped over and fell down_comma_and the stu...
997,embarrassed,I feel down in front of my students during games.,oof that sounds quite embarrassing_comma_ hope...
998,afraid,I feel scared of the future because it's so un...,Do you ever feel afraid?


In [55]:
# use the first 100 rows of the dataset for training
training_df = df_train

# apply the prepare_example_conversation function to each row of the training_df
training_data = training_df.apply(prepare_example_conversation, axis=1).tolist()

for example in training_data[:20]:
    print(example)

{'messages': [{'role': 'system', 'content': 'You are an empathetic assistant. You are to generate empathetic responses to prompts.'}, {'role': 'user', 'content': 'Title: sentimental\n\nprompt: I remember going to the fireworks with my best friend. There was a lot of people_comma_ but it only felt like us in the world.\n\nutterance: I remember going to see the fireworks with my best friend. It was the first time we ever spent time alone together. Although there was a lot of people_comma_ we felt like the only people in the world. '}, {'role': 'assistant', 'content': 'I remember going to the fireworks with my best friend. There was a lot of people_comma_ but it only felt like us in the world.'}]}
{'messages': [{'role': 'system', 'content': 'You are an empathetic assistant. You are to generate empathetic responses to prompts.'}, {'role': 'user', 'content': 'Title: sentimental\n\nprompt: I remember going to the fireworks with my best friend. There was a lot of people_comma_ but it only fel

In [56]:
validation_df = df_validation
validation_data = validation_df.apply(prepare_example_conversation, axis=1).tolist()

In [57]:
def write_jsonl(data_list: list, filename: str) -> None:
    with open(filename, "w") as out:
        for ddict in data_list:
            jout = json.dumps(ddict) + "\n"
            out.write(jout)

In [58]:
training_file_name = "tmp_recipe_finetune_training.jsonl"
write_jsonl(training_data, training_file_name)

validation_file_name = "tmp_recipe_finetune_validation.jsonl"
write_jsonl(validation_data, validation_file_name)

In [59]:
# print the first 5 lines of the training file
!head -n 5 tmp_recipe_finetune_training.jsonl

{"messages": [{"role": "system", "content": "You are an empathetic assistant. You are to generate empathetic responses to prompts."}, {"role": "user", "content": "Title: sentimental\n\nprompt: I remember going to the fireworks with my best friend. There was a lot of people_comma_ but it only felt like us in the world.\n\nutterance: I remember going to see the fireworks with my best friend. It was the first time we ever spent time alone together. Although there was a lot of people_comma_ we felt like the only people in the world. "}, {"role": "assistant", "content": "I remember going to the fireworks with my best friend. There was a lot of people_comma_ but it only felt like us in the world."}]}
{"messages": [{"role": "system", "content": "You are an empathetic assistant. You are to generate empathetic responses to prompts."}, {"role": "user", "content": "Title: sentimental\n\nprompt: I remember going to the fireworks with my best friend. There was a lot of people_comma_ but it only fel

In [60]:
import json

training_file_name = "tmp_recipe_finetune_training.jsonl"
validation_file_name = "tmp_recipe_finetune_validation.jsonl"

def prepare_data(dictionary_data, final_file_name):
    with open(final_file_name, 'w') as outfile:
        for entry in dictionary_data:
        	json.dump(entry, outfile)
        	outfile.write('\n')

prepare_data(training_data, "training_data.jsonl")
prepare_data(validation_data, "validation_data.jsonl")

In [61]:
training_file_id = openai.files.create(
  file=open(training_file_name, "rb"),
  purpose="fine-tune"
)

validation_file_id = openai.files.create(
  file=open(validation_file_name, "rb"),
  purpose="fine-tune"
)

print(f"Training File ID: {training_file_id}")
print(f"Validation File ID: {validation_file_id}")

Training File ID: FileObject(id='file-JnqjxJVKWfHWtyGGRtOyV790', bytes=4312706, created_at=1706493518, filename='tmp_recipe_finetune_training.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None)
Validation File ID: FileObject(id='file-1EsyVcsO4zhAczIlHniRZoDM', bytes=497664, created_at=1706493519, filename='tmp_recipe_finetune_validation.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None)


In [62]:
response = openai.fine_tuning.jobs.create(
  training_file=training_file_id.id,
  validation_file=validation_file_id.id,
  model="gpt-3.5-turbo",
  hyperparameters={
    "n_epochs": 15,
	"batch_size": 3,
	"learning_rate_multiplier": 0.3
  }
)
job_id = response.id
status = response.status

print(f'Fine-tunning model with jobID: {job_id}.')
print(f"Training Response: {response}")
print(f"Training Status: {status}")

Fine-tunning model with jobID: ftjob-m1AEdyOPa5btfvDCsj4Zts68.
Training Response: FineTuningJob(id='ftjob-m1AEdyOPa5btfvDCsj4Zts68', created_at=1706493520, error=None, fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs=15, batch_size=3, learning_rate_multiplier=0.3), model='gpt-3.5-turbo-0613', object='fine_tuning.job', organization_id='org-LHaiholBKWEH7fvxm02ReYJZ', result_files=[], status='validating_files', trained_tokens=None, training_file='file-JnqjxJVKWfHWtyGGRtOyV790', validation_file='file-1EsyVcsO4zhAczIlHniRZoDM')
Training Status: validating_files


In [ ]:
import time

status = openai.fine_tuning.jobs.retrieve(job_id).status
if status not in ["succeeded", "failed"]:
    print(f"Job not in terminal status: {status}. Waiting.")
    while status not in ["succeeded", "failed"]:
        time.sleep(100)
        status = openai.fine_tuning.jobs.retrieve(job_id).status
        print(f"Status: {status}")
else:
    print(f"Finetune job {job_id} finished with status: {status}")
print("Checking other finetune jobs in the subscription.")
result = openai.fine_tuning.jobs.list()
print(f"Found {len(result.data)} finetune jobs.")

Job not in terminal status: validating_files. Waiting.
Status: validating_files
Status: validating_files
Status: validating_files
Status: validating_files
Status: validating_files
Status: validating_files
Status: validating_files
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
St

In [ ]:
# Retrieve the finetuned model
fine_tuned_model = result.data[0].fine_tuned_model
print(fine_tuned_model)

In [ ]:
def set_open_params(
    model=fine_tuned_model,
    temperature=0.7,
    max_tokens=256,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0,
):
    """ set openai parameters"""

    openai_params = {}

    openai_params['model'] = model
    openai_params['temperature'] = temperature
    openai_params['max_tokens'] = max_tokens
    openai_params['top_p'] = top_p
    openai_params['frequency_penalty'] = frequency_penalty
    openai_params['presence_penalty'] = presence_penalty
    return openai_params

def get_completion(params, messages):
    """ GET completion from openai api"""

    response = openai.chat.completions.create(
        model = params['model'],
        messages = messages,
        temperature = params['temperature'],
        max_tokens = params['max_tokens'],
        top_p = params['top_p'],
        frequency_penalty = params['frequency_penalty'],
        presence_penalty = params['presence_penalty'],
    )
    return response

In [ ]:

params = set_open_params()

new_prompt = "how to make a person happy when he feels lonely"

messages = [
    {
        "role": "user",
        "content": new_prompt
    }
]

response = get_completion(params, messages)


In [ ]:

response.choices[0].message.content